In [1]:
# Import dependencies and initialize findspark
import os
# Find the latest version of spark 3.x  from https://downloads.apache.org/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark
!pip install hvplot

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

from pyspark import SparkFiles

import pandas as pd
import tensorflow as tf
import h5py

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 233 kB in 2s (112 kB/s)
Reading package lists... Done


In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("energy_weather").getOrCreate()

/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(


In [3]:
# Read in data from Github
from pyspark import SparkFiles
import requests

url = "https://raw.githubusercontent.com/fraserclaire/Group3_Project4/main/Resources/energy_weather_severity_df.csv"
response = requests.get(url)

with open("energy_weather_severity_df.csv", "wb") as f:
    f.write(response.content)


spark.sparkContext.addFile(url)
weather_energy_df = spark.read.csv(SparkFiles.get('energy_weather_severity_df.csv'), inferSchema=True, header=True)


# Create a view for weather dataset
weather_energy_df.createOrReplaceTempView('energy_weather')
spark.sql("""SELECT * FROM  energy_weather""").show()


+---+------------------+-------------------+---------+------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-------------------+---------

In [4]:
# Convert the PySpark DataFrame to a pandas DataFrame
weather_energy_pandas = weather_energy_df.toPandas()
weather_energy_pandas = weather_energy_pandas.drop(columns=['_c0','region', 'sub-region', 'country'])
weather_energy_pandas.head()

,latitude,longitude,daylight_hr_avg_2011,daylight_hr_avg_2012,daylight_hr_avg_2013,daylight_hr_avg_2014,daylight_hr_avg_2015,daylight_hr_avg_2016,daylight_hr_avg_2017,daylight_hr_avg_2018,...,2017-TR_Severity,2018-Fossil_Divided,2018-TR_Divided,2018-TR_Severity,2019-Fossil_Divided,2019-TR_Divided,2019-TR_Severity,2020-Fossil_Divided,2020-TR_Divided,2020-TR_Severity
0,32.794376,-86.809296,12.170882,12.168212,12.170964,12.170877,12.170845,12.168266,12.170945,12.170905,...,1,0.408463,0.591537,1,0.407242,0.592758,1,0.387801,0.612199,1
1,61.335674,-152.257278,12.399331,12.390161,12.399591,12.399296,12.399188,12.390317,12.399516,12.399377,...,0,0.698536,0.301464,0,0.711359,0.288641,0,0.706481,0.293519,0
2,33.708260,-111.354034,12.173668,12.170898,12.173753,12.173663,12.173630,12.170956,12.173734,12.173692,...,1,0.478514,0.521486,1,0.490770,0.509230,0,0.465072,0.534928,1
3,34.973637,-92.401520,12.177714,12.174804,12.177804,12.177709,12.177674,12.174863,12.177784,12.177739,...,1,0.475234,0.524766,1,0.450444,0.549556,1,0.450958,0.549042,1
4,36.098415,-119.649811,12.181508,12.178467,12.181603,12.181503,12.181466,12.178529,12.181581,12.181534,...,1,0.324942,0.675058,1,0.297403,0.702597,1,0.288082,0.711918,1


In [5]:
y = weather_energy_pandas["2020-TR_Severity"]
X = weather_energy_pandas.drop(columns="2020-TR_Severity")

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=1,
                                                    stratify=y)

# Create a Logistic Regression Model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', random_state=1)

# Fit (train) or model using the training data
classifier.fit(X_train, y_train)

predictions = classifier.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).tail(10)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,Prediction,Actual
41,1,1
24,0,0
49,1,1
20,0,0
32,1,1
18,0,0
48,0,0
40,0,1
16,1,1
25,0,0


In [7]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.9736842105263158
Testing Data Score: 0.9230769230769231


In [8]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[6, 0],
       [1, 6]])

In [10]:
from sklearn.metrics import classification_report
target_names = ["High", "Low"]
print(classification_report(y_test, predictions, target_names=target_names))

              precision    recall  f1-score   support

        High       0.86      1.00      0.92         6
         Low       1.00      0.86      0.92         7

    accuracy                           0.92        13
   macro avg       0.93      0.93      0.92        13
weighted avg       0.93      0.92      0.92        13

